In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# changed animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from crud import Animal_Shelter

###########################
# Data Manipulation / Model
###########################
# updated with my username and password and CRUD Python module name

username = "aacuser"
password = "skibolski"
host = "nv-desktop-services.apporto.com"
port = "30709"
db = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = Animal_Shelter(username, password, host, port, db, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Added in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Placed the HTML image tag in the line below into the app.layout code according to my design
#included a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('CS-340 Dashboard: Nii Amatey Tagoe'))),
    html.Hr(),
    html.Div(
        #Added code for the interactive filtering option
        children=[
            html.Div(
                className='filter-options',
                style={'text-align': 'center'},
                children=[
                    html.Label('Select Rescue Type:'),
                    dcc.RadioItems(
                        id='filter-type',
                        options=[
                            {'label': 'Water Rescue', 'value': 'Water Rescue'},
                            {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                            {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                            {'label': 'Reset', 'value': 'Reset'},
                        ],
                        value='Reset',  # Default value
                        labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                    ),
                ]
            ),
        ],
    ),
          
    html.Hr(),
    # Setting up the features for my interactive data table to make it user-friendly for my client
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
            'whiteSpace': 'normal'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current=0,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that my chart and my geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {}

    if filter_type == 'Water Rescue':
        # Water Rescue: Dogs that are Labrador Retriever Mix or Chesapeake Bay Retriever, age between 26 and 156 weeks
        query = {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Mountain Rescue':
        # Mountain Rescue: Dogs that are German Shepherd, Alaskan Malamute, or other specified breeds, age between 26 and 156 weeks
        query = {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                   "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Disaster Rescue':
        # Disaster Rescue or Individual Tracking: Dogs that are Golden Retriever,German shepherd, Bloodhound and Rottweilers, age between 20 and 300 weeks
        query = {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                                   "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    else:
        # Reset: Return all records
        data = df.to_dict('records')
        return data

    # Query the database using my CRUD module
    filtered_records = db.read(query)
    filtered_df = pd.DataFrame.from_records(filtered_records)
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

    # Ensures 'age_upon_outcome_in_weeks' is in the DataFrame
    if 'age_upon_outcome_in_weeks' not in filtered_df.columns:
        filtered_df['age_upon_outcome_in_weeks'] = calculate_age_in_weeks(filtered_df['age_upon_outcome'])

    data = filtered_df.to_dict('records')
    return data
    ## Added code to filter interactive data table with MongoDB queries

     
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # added code for chart of your choice (e.g. pie chart)
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available to display.")]
    else:
        dff = pd.DataFrame.from_dict(viewData)

        # Group the data by breed and count occurrences
        breed_counts = dff['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']

        # Create a bar chart with specified size
        fig = px.bar(
            breed_counts,
            x='breed',
            y='count',
            title='Breed Distribution',
            height=500,  # Adjust height here
            width=800    # Adjust width here
        )

        # Update layout for better readability
        fig.update_layout(
            xaxis_title="Breed",
            yaxis_title="Count",
            xaxis_tickangle=-45
        )

        return [
            dcc.Graph(figure=fig)
        ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index): 
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or index == []:
        row = 0
    else:
        row = index[0]
    # Get the latitude and longitude
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        # Ensures that lat and lon are valid numbers
        if pd.isnull(lat) or pd.isnull(lon):
            lat = 30.75
            lon = -97.48
    else:
        # Default to Austin, TX coordinates
        lat = 30.75
        lon = -97.48
    # Get breed and name
    breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else 'Unknown'
    name = dff.iloc[row]['name'] if 'name' in dff.columns else 'Unknown'

    # Return the map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[lat, lon],
                             children=[
                                 dl.Tooltip(breed),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(name)
                                 ])
                             ])
               ])
    ]




app.run_server(debug=True)


Dash app running on http://127.0.0.1:26407/
